In [1]:
import os
#os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/local/cuda-10.0/nvvm/libdevice"
#os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-10.0/nvvm/lib64/libnvvm.so"

import pandas as pd
import regex as re
from tqdm.notebook import tqdm
import PIL
import PIL.Image
import os
import pandas as pd
import regex as re
from numba import cuda
import math
from tqdm.notebook import tqdm
import matplotlib
import matplotlib.pyplot as plt
import math
import numpy as np
from scipy.ndimage.filters import gaussian_filter
from tqdm.notebook import tqdm
%matplotlib inline

In [2]:
def fixation_detection(x_values, y_values, time, maxdist=25, mindur=0.020):

    """Detects fixations, defined as consecutive samples with an inter-sample
    distance of less than a set amount of pixels (disregarding missing data)

    arguments

    x-numpy array of x positions
    y-numpy array of y positions
    time-numpy array of EyeTribe timestamps

    keyword arguments

    missing - value to be used for missing data (default = 0.0)
    maxdist - maximal inter sample distance in pixels (default = 25)
    mindur - minimal duration of a fixation in milliseconds; detected
    fixation cadidates will be disregarded if they are below
    this duration (default = 100)

    returns
    Sfix, Efix

    Sfix - list of lists, each containing [starttime]
    Efix - list of lists, each containing [starttime, endtime, duration, endx, endy]
    """

    # empty list to contain data
    Sfix = []
    Efix = []
    
    # loop through all coordinates
    si = 0
    fixstart = False
    for i in range(1,len(x_values)):
        
        # calculate Euclidean distance from the current fixation coordinate
        # to the next coordinate
        
        squared_distance = ((x_values[si]-x_values[i])**2 + (y_values[si]-y_values[i])**2)
        dist = 0.0
        if squared_distance > 0:
            dist = squared_distance**0.5
            
        # check if the next coordinate is below maximal distance
        
        if dist <= maxdist:
            # start a new fixation
            si = 0 + i
            fixstart = False
            Sfix.append([time[i]])
        elif dist > maxdist and not fixstart:
            
            # end the current fixation
            
            fixstart = False
            if Sfix:
                

                # only store the fixation if the duration is ok

                if time[i-1]-Sfix[-1][0] >= mindur:
                    Efix.append([Sfix[-1][0], time[i-1], time[i-1]-Sfix[-1][0], x_values[si], y_values[si]])

                # delete the last fixation start if it was too short

                else:
                    Sfix.pop(-1)
                si = 0 + i
            else:
                pass
        elif not fixstart:
            si += 1
            
    #add last fixation end (we can lose it if dist > maxdist is false for the last point)
    
        elif len(Sfix) > len(Efix):
            Efix.append([Sfix[-1][0], time[len(x_values)-1], time[len(x_values)-1]-Sfix[-1][0], x_values[si], y_values[si]])
        
    return Sfix, Efix

def saccade_detection(x_values, y_values, time, minlen=0.020, maxvel=20, maxacc=200):
    """Detects saccades, defined as consecutive samples with an inter-sample
    velocity of over a velocity threshold or an acceleration threshold

    arguments

    x		-	numpy array of x positions
    y		-	numpy array of y positions
    time		-	numpy array of tracker timestamps in milliseconds

    keyword arguments

    missing	-	value to be used for missing data (default = 0.0)
    minlen	-	minimal length of saccades in milliseconds; all detected
                saccades with len(sac) < minlen will be ignored
                (default = 5)
    maxvel	-	velocity threshold in pixels/second (default = 40)
    maxacc	-	acceleration threshold in pixels / second**2
                (default = 340)

    returns
    Ssac, Esac
            Ssac	-	list of lists, each containing [starttime]
            Esac	-	list of lists, each containing [starttime, endtime, duration, startx, starty, endx, endy]
    """

    # CONTAINERS
    Ssac = []
    Esac = []

    # INTER-SAMPLE MEASURES
    # the distance between samples is the square root of the sum
    # of the squared horizontal and vertical interdistances
    intdist = (np.diff(x_values) ** 2 + np.diff(y_values) ** 2) ** 0.5
    # get inter-sample times
    inttime = np.diff(time)
    # recalculate inter-sample times to seconds
    inttime =inttime / 1000

    # VELOCITY AND ACCELERATION
    # the velocity between samples is the inter-sample distance
    # divided by the inter-sample time
    vel = intdist / inttime
    # the acceleration is the sample-to-sample difference in
    # eye movement velocity
    acc = np.diff(vel)

    # SACCADE START AND END
    t0i = 0
    stop = False
    while not stop:
        # saccade start (t1) is when the velocity or acceleration
        # surpass threshold, saccade end (t2) is when both return
        # under threshold

        # detect saccade starts
        sacstarts = np.where((vel[1 + t0i:] > maxvel).astype(int) + (acc[t0i:] > maxacc).astype(int) >= 1)[0]
        if len(sacstarts) > 0:
            # timestamp for starting position
            t1i = t0i + sacstarts[0] + 1
            if t1i >= len(time) - 1:
                t1i = len(time) - 2
            t1 = time[t1i]

            # add to saccade starts
            Ssac.append([t1])

            # detect saccade endings
            sacends = np.where((vel[1 + t1i:] < maxvel).astype(int) + (acc[t1i:] < maxacc).astype(int) == 2)[0]
            if len(sacends) > 0:
                # timestamp for ending position
                t2i = sacends[0] + 1 + t1i + 2
                if t2i >= len(time):
                    t2i = len(time) - 1
                t2 = time[t2i]
                dur = t2 - t1

                # ignore saccades that did not last long enough
                if dur >= minlen:
                    # add to saccade ends
                    Esac.append([t1, t2, dur, x_values[t1i], y_values[t1i], x_values[t2i], y_values[t2i]])
                else:
                    # remove last saccade start on too low duration
                    Ssac.pop(-1)

                # update t0i
                t0i = 0 + t2i
            else:
                stop = True
        else:
            stop = True

    return Ssac, Esac

COLS = {"butter": ['#fce94f','#edd400','#c4a000'],
        "orange": ['#fcaf3e','#f57900','#ce5c00'],
        "chocolate": ['#e9b96e','#c17d11','#8f5902'],
        "chameleon": ['#8ae234','#73d216','#4e9a06'],
        "skyblue": ['#729fcf','#3465a4','#204a87'],
        "plum": ['#ad7fa8','#75507b','#5c3566'],
        "scarletred":['#ef2929','#cc0000','#a40000'],
        "aluminium": ['#eeeeec','#d3d7cf','#babdb6','#888a85','#555753','#2e3436']}
# FONT
FONT = {'family': 'Ubuntu', 'size': 12}
matplotlib.rc('font', **FONT)

def draw_display(dispsize, imagefile=True):
    """Returns a matplotlib.pyplot Figure and its axes, with a size of
    dispsize, a black background colour, and optionally with an image drawn
    onto it
    arguments
    dispsize		-	tuple or list indicating the size of the display,
                    e.g. (1024,768)
    keyword arguments
    imagefile		-	full path to an image file over which the heatmap
                    is to be laid, or None for no image; NOTE: the image
                    may be smaller than the display size, the function
                    assumes that the image was presented at the centre of
                    the display (default = None)
    returns
    fig, ax		-	matplotlib.pyplot Figure and its axes: field of zeros
                    with a size of dispsize, and an image drawn onto it
                    if an imagefile was passed
    """

    # construct screen (black background)
    screen = np.zeros((dispsize[1], dispsize[0], 4), dtype='float32')
    # if an image location has been passed, draw the image
    if imagefile != None:
        # check if the path to the image exists
        #if not os.path.isfile(imagefile):
            #raise Exception("ERROR in draw_display: imagefile not found at '%s'" % imagefile)
        # load image
        #img = image.imread(imagefile)
        img = plt.imread(imagefile)
        #img = PIL.Image.open(imagefile)

        # width and height of the image
        w, h = len(img[0]), len(img)
        #w,h=img.size
        # x and y position of the image on the display
        x = dispsize[0] // 2 - w // 2
        y = dispsize[1] // 2 - h // 2
        # draw the image on the screen
        screen[y:y + h, x:x + w, :] += img
    # dots per inch
    dpi = 100.0
    # determine the figure size in inches
    figsize = (dispsize[0] / dpi, dispsize[1] / dpi)
    # create a figure
    fig = plt.figure(figsize=figsize, dpi=dpi, frameon=False)
    ax = plt.Axes(fig, [0, 0, 1, 1])
    ax.set_axis_off()
    fig.add_axes(ax)
    # plot display
    ax.axis([0, dispsize[0], 0, dispsize[1]])
    ax.imshow(screen)  # , origin='upper')

    return fig ,ax

def parse_fixations(fixations):
    """Returns all relevant data from a list of fixation ending events

    arguments

    fixations		-	a list of fixation ending events from a single trial,
                    as produced by edfreader.read_edf, e.g.
                    edfdata[trialnr]['events']['Efix']

    returns

    fix		-	a dict with three keys: 'x', 'y', and 'dur' (each contain
                a numpy array) for the x and y coordinates and duration of
                each fixation
    """

    # empty arrays to contain fixation coordinates
    fix = {'x': np.zeros(len(fixations)),
           'y': np.zeros(len(fixations)),
           'dur': np.zeros(len(fixations))}
    # get all fixation coordinates
    for fixnr in range(len(fixations)):
        stime, etime, dur, ex, ey = fixations[fixnr]
        fix['x'][fixnr] = ex
        fix['y'][fixnr] = ey
        fix['dur'][fixnr] = dur

    return fix

def draw_fixations(fixations, dispsize, imagefile=True, durationsize=False, durationcolour=False, alpha=0.5,
                   savefilename=True):
    
    """Draws circles on the fixation locations, optionally on top of an image,
    with optional weigthing of the duration for circle size and colour

    arguments

    fixations		-	a list of fixation ending events from a single trial,
                    as produced by edfreader.read_edf, e.g.
                    edfdata[trialnr]['events']['Efix']
    dispsize		-	tuple or list indicating the size of the display,
                    e.g. (1024,768)

    keyword arguments

    imagefile		-	full path to an image file over which the heatmap
                    is to be laid, or None for no image; NOTE: the image
                    may be smaller than the display size, the function
                    assumes that the image was presented at the centre of
                    the display (default = None)
    durationsize	-	Boolean indicating whether the fixation duration is
                    to be taken into account as a weight for the circle
                    size; longer duration = bigger (default = True)
    durationcolour	-	Boolean indicating whether the fixation duration is
                    to be taken into account as a weight for the circle
                    colour; longer duration = hotter (default = True)
    alpha		-	float between 0 and 1, indicating the transparancy of
                    the heatmap, where 0 is completely transparant and 1
                    is completely untransparant (default = 0.5)
    savefilename	-	full path to the file in which the heatmap should be
                    saved, or None to not save the file (default = None)

    returns

    fig			-	a matplotlib.pyplot Figure instance, containing the
                    fixations
    """
    fig, ax = draw_display(dispsize, imagefile=imagefile)
    for i in range(len(fixations)):
        
        
       
    # FIXATIONS
        fix = parse_fixations([fixations[i]])

    # IMAGE
        

    # CIRCLES
    # duration weigths
        if durationsize:
            #siz = 1 * (fix['dur'] / 30.0)
            siz = round(round(1 * (fix['dur'],5)) * 10000)
        else:
            #siz = 1 * np.median(fix['dur'] / 30.0)
            siz = round(round(1 * np.median(fix['dur']),5) * 10000)
            # if siz> 450.:
                    #ax.scatter(fix['x'], fix['y'], s=siz, c=col, marker='o', cmap='jet', alpha=alpha, edgecolors='none')

        if durationcolour:
            col = fix['dur']
        else:
            col = COLS['chameleon'][2]
        # draw circles
        ax.scatter(fix['x'], fix['y'], s=siz, c=col, marker='o', cmap='jet', alpha=alpha, edgecolors='none')
        if siz > 450:
            ax.annotate(str(siz),(fix['x'],fix['y']),color='black', ha='center')

        # FINISH PLOT
        # invert the y axis, as (0,0) is top left on a display
    ax.invert_yaxis()
        # save the figure if a file name was provided
    if savefilename != None:
        fig.savefig(savefilename)

    return fig

def draw_scanpath(fixations, saccades, dispsize, imagefile=True, alpha=0.1, savefilename=True):
    
    """Draws a scanpath: a series of arrows between numbered fixations,
    optionally drawn over an image

    arguments

    fixations		-	a list of fixation ending events from a single trial,
                    as produced by edfreader.read_edf, e.g.
                    edfdata[trialnr]['events']['Efix']
    saccades		-	a list of saccade ending events from a single trial,
                    as produced by edfreader.read_edf, e.g.
                    edfdata[trialnr]['events']['Esac']
    dispsize		-	tuple or list indicating the size of the display,
                    e.g. (1024,768)

    keyword arguments

    imagefile		-	full path to an image file over which the heatmap
                    is to be laid, or None for no image; NOTE: the image
                    may be smaller than the display size, the function
                    assumes that the image was presented at the centre of
                    the display (default = None)
    alpha		-	float between 0 and 1, indicating the transparancy of
                    the heatmap, where 0 is completely transparant and 1
                    is completely untransparant (default = 0.5)
    savefilename	-	full path to the file in which the heatmap should be
                    saved, or None to not save the file (default = None)

    returns

    fig			-	a matplotlib.pyplot Figure instance, containing the
                    heatmap
    """

    # image
    fig,ax  = draw_display(dispsize, imagefile=imagefile)
    Fixt = []
    # FIXATIONS
    # parse fixations
    for i in range(len(fixations)):
        
        fix = parse_fixations([fixations[i]])
        Fixt.append(fix)
        # draw fixations
        ax.scatter(fix['x'], fix['y'], s=round(round(1 * np.median(fix['dur']),5) * 10000), c=COLS['chameleon'][2], marker='o', cmap='jet',
                   alpha=alpha, edgecolors='none')
        # draw annotations (fixation numbers)
    for i in range(len(Fixt)):

        ax.annotate(str(i + 1), (Fixt[i]['x'], Fixt[i]['y']), color='Red', alpha=1,
                    horizontalalignment='center', verticalalignment='center') #multialignment='center')
    #for j in range(len(Fixt)):
        x_va = []
        y_va = []
        for i in range(len(Fixt)):
            x_va.append(Fixt[i]['x'][0])
            y_va.append(Fixt[i]['y'][0])
            
        different_x = []
        different_y = []
        for i in range(1,len(Fixt)):
            a = x_va[i] - x_va[i-1]
            b = y_va[i] - y_va[i-1]
            different_x.append(a)
            different_y.append(b)
        different_x.append(0.0)
        different_y.append(0.0)
        for k in range(len(x_va)):
            
            ax.arrow(x_va[k],y_va[k],different_x[k],different_y[k],fc=COLS['aluminium'][0], ec= None, fill=True,
                     shape='full', width=5, head_width=8,alpha=alpha, head_starts_at_zero=False, overhang=0)


    # SACCADES
        #if saccades:
            #for j in range(len(saccades)):
                
            # loop through all saccades
                #for st, et, dur, sx, sy, ex, ey in [saccades[j]]:
                    # draw an arrow between every saccade start and ending
                    #ax.arrow(sx, sy, ex - sx, ey - sy, alpha=alpha, fc=COLS['aluminium'][0], ec=COLS['aluminium'][5], fill=True,
                             #shape='full', width=10, head_width=20, head_starts_at_zero=False, overhang=0)

    # invert the y axis, as (0,0) is top left on a display
    ax.invert_yaxis()
    # save the figure if a file name was provided
    if savefilename != None:
        fig.savefig(savefilename)

    return fig,Fixt

def gen_heatmap(click_data, width, height, radius):
    heatmap = np.zeros((height, width), dtype=np.int32)
    mask = np.ones((height, width), dtype=np.int32)

    for x, y in click_data:
        Y, X = np.ogrid[:height, :width]
        dist_from_center = np.sqrt((X - x)**2 + (Y-y)**2)

        current_mask = dist_from_center < radius
        heatmap[current_mask] = heatmap[current_mask] + mask[current_mask]
    return heatmap

#def heatmap_cuda(click_data, width, height, radius):
    #threadsperblock = (32, 32)
    #blockspergrid_x = int(np.ceil(width / threadsperblock[0]))
    #blockspergrid_y = int(np.ceil(height / threadsperblock[1]))
    #blockspergrid = (blockspergrid_y, blockspergrid_x)

    #heatmap = np.zeros((height, width), dtype=np.int32)
    #stream = cuda.stream()
    #heatmap = cuda.to_device(heatmap, stream=stream)

    #WIDTH = int(width)
    #HEIGHT = int(height)
    #RADIUS = float(radius)

    #@cuda.jit
    #def circular_addition(heatmap, center_x, center_y):
        #y, x = cuda.grid(2)

        #if center_x - radius <= x <= center_x + radius and center_y - radius <= y <= center_y + radius:
            #if y < HEIGHT and x < WIDTH:
                #distance = ((center_x - x) ** 2 + (center_y - y) ** 2) ** 0.5
                #if distance <= RADIUS:
                    #heatmap[y, x] += 1

    #for x, y in click_data:
        #circular_addition[blockspergrid, threadsperblock](heatmap, x, y)

    #return heatmap.copy_to_host(stream=stream)

def calculateAOIvalues(x_values, y_values, time, x1, y1, x2, y2):
  firstTimeInAOI = 0
  startTime = 0
  firstPassDuration = 0
  secondPassDuration = 0
  totalDuration = 0
  revisits = 0
  timeInAOI = 0
  revisitFlag = True
  AOIflag = False
  values = {}
  startIndex = 0
  
  'no of fixation points in AOI'

  for i in range(1,len(x_values)):
    if x1 <= x_values[i] <= x2 and y1 <= y_values[i] <= y2:
      AOIflag = True
      if firstTimeInAOI == 0:
        firstTimeInAOI = time[i] - time[0]
      if startTime == 0:
        startTime = time[i]

      #timeInAOI = time [i] - startTime

      if revisitFlag:
        revisits += 1
        revisitFlag = False

      if(startIndex == 0):
        startIndex = i

    elif AOIflag:
      AOIflag = False
      totalDuration = totalDuration + (time [i] - startTime)
      startTime = 0
      revisitFlag = True
      if firstPassDuration == 0:
        firstPassDuration = time[i] - time[startIndex]
      elif secondPassDuration == 0:
        secondPassDuration = time[i] - time[startIndex]
      startIndex = 0

  values["firstTimeInAOI"] = firstTimeInAOI
  values["firstPassDuration"] = firstPassDuration
  values["secondPassDuration"] = secondPassDuration
  values["totalDuration"] = totalDuration
  values["revisits"] = revisits
  return values


def calculateAOIvalueswithFixations(Efix, x1, y1, x2, y2):
  firstPassDuration = 0
  secondPassDuration = 0
  totalDuration = 0
  revisits = 0
  noOfFixtionsInAOI = 0
  revisitFlag = True
  lastFix = False
  values = {}
  startIndex = 0
  
  'no of fixation points in AOI'

  for idx, row in Efix.iterrows():
    if x1 <= row['endx'] <= x2 and y1 <= row['endy'] <= y2:
      noOfFixtionsInAOI += 1
      if lastFix == False:
        lastFix = True
        revisits += 1
      totalDuration = totalDuration + row['duration']
      if firstPassDuration == 0:
        firstPassDuration = row['duration']
      elif secondPassDuration == 0:
        secondPassDuration = row['duration']
    else:
      lastFix = False

  values["firstPassDuration"] = firstPassDuration
  values["secondPassDuration"] = secondPassDuration
  values["totalDuration"] = totalDuration
  values["revisits"] = revisits
  values["noOfFixtionsInAOI"] = noOfFixtionsInAOI
  return values

In [ ]:
#sessionNo = ["001","001","001","001","001","001","001","001","001","001","001","001","001","006","007","007","001","001","001","001","001"]
#userInfo = ["user1","user2","user3","user4","user5","user6","user7","user8","user9","user10","user11","user12","user13","user14","user15","user16","user17","user18","user19","user20","user21"]
userInfo = ["user8"]
sessionNo = ["001"]


screen_width = 1920
screen_height = 1080
dimension = (1080,1920)
number_of_clicks = 1000

AOIs = {'ArraySum(WOC)': [{'x1': 600, 'x2': 643, 'y1': 268, 'y2': 324},
  {'x1': 1154, 'x2': 1187, 'y1': 348, 'y2': 405},
  {'x1': 476, 'x2': 521, 'y1': 559, 'y2': 620}],
 'ArraySumresp(WOC)': [{'x1': 602, 'x2': 641, 'y1': 270, 'y2': 320},
  {'x1': 1154, 'x2': 1194, 'y1': 353, 'y2': 409},
  {'x1': 474, 'x2': 520, 'y1': 564, 'y2': 621},
  {'x1': 1297, 'x2': 1343, 'y1': 648, 'y2': 702},
  {'x1': 468, 'x2': 521, 'y1': 772, 'y2': 829},
  {'x1': 396, 'x2': 439, 'y1': 815, 'y2': 872}],
 'ArraySum(WIC)': [{'x1': 600, 'x2': 646, 'y1': 248, 'y2': 360},
  {'x1': 1151, 'x2': 1197, 'y1': 332, 'y2': 389},
  {'x1': 1216, 'x2': 1259, 'y1': 417, 'y2': 473},
  {'x1': 556, 'x2': 603, 'y1': 504, 'y2': 552},
  {'x1': 474, 'x2': 517, 'y1': 586, 'y2': 643}],
 'ArraySumresp(WIC)': [{'x1': 1152, 'x2': 1199, 'y1': 333, 'y2': 386},
  {'x1': 1212, 'x2': 1261, 'y1': 423, 'y2': 467},
  {'x1': 554, 'x2': 601, 'y1': 501, 'y2': 552},
  {'x1': 468, 'x2': 521, 'y1': 580, 'y2': 644},
  {'x1': 1294, 'x2': 1341, 'y1': 669, 'y2': 724},
  {'x1': 475, 'x2': 520, 'y1': 493, 'y2': 855},
  {'x1': 392, 'x2': 440, 'y1': 835, 'y2': 891}],
 'ArraySum(CNL)': [{'x1': 551, 'x2': 594, 'y1': 254, 'y2': 306},
  {'x1': 608, 'x2': 656, 'y1': 337, 'y2': 373},
  {'x1': 671, 'x2': 716, 'y1': 441, 'y2': 478},
  {'x1': 668, 'x2': 713, 'y1': 506, 'y2': 552},
  {'x1': 606, 'x2': 652, 'y1': 575, 'y2': 622}],
 'ArraySumresp(CNL)': [{'x1': 553, 'x2': 588, 'y1': 264, 'y2': 303},
  {'x1': 606, 'x2': 651, 'y1': 336, 'y2': 374},
  {'x1': 671, 'x2': 713, 'y1': 436, 'y2': 486},
  {'x1': 670, 'x2': 408, 'y1': 509, 'y2': 551},
  {'x1': 612, 'x2': 650, 'y1': 574, 'y2': 625},
  {'x1': 610, 'x2': 653, 'y1': 686, 'y2': 726},
  {'x1': 606, 'x2': 650, 'y1': 787, 'y2': 839},
  {'x1': 550, 'x2': 588, 'y1': 822, 'y2': 871}],
 'BubbleSort(WOC)': [{'x1': 794, 'x2': 836, 'y1': 193, 'y2': 464},
  {'x1': 1111, 'x2': 1156, 'y1': 264, 'y2': 321},
  {'x1': 1235, 'x2': 1283, 'y1': 414, 'y2': 464},
  {'x1': 503, 'x2': 553, 'y1': 557, 'y2': 613},
  {'x1': 438, 'x2': 484, 'y1': 596, 'y2': 646}],
 'BubbleSortresp(WOC)': [{'x1': 798, 'x2': 833, 'y1': 196, 'y2': 250},
  {'x1': 1114, 'x2': 1154, 'y1': 270, 'y2': 317},
  {'x1': 509, 'x2': 548, 'y1': 560, 'y2': 607},
  {'x1': 443 , 'x2': 480, 'y1': 596, 'y2': 646},
  {'x1': 442, 'x2': 480, 'y1': 812, 'y2': 866},
  {'x1': 372, 'x2': 409, 'y1': 849, 'y2': 899},
  {'x1': 1239, 'x2': 1279, 'y1': 410, 'y2': 167},
  {'x1': 1134, 'x2': 1169, 'y1': 671, 'y2': 711}],
 'BubbleSort(WIC)': [{'x1': 807, 'x2': 845, 'y1': 184, 'y2': 231},
  {'x1': 1105, 'x2': 1143, 'y1': 253, 'y2': 299},
  {'x1': 1040, 'x2': 1074, 'y1': 323, 'y2': 362},
  {'x1': 1205, 'x2': 1241, 'y1': 352, 'y2': 402},
  {'x1': 1223, 'x2': 1254, 'y1': 394, 'y2': 433},
  {'x1': 670, 'x2': 709, 'y1': 523, 'y2': 570},
  {'x1': 605, 'x2': 643, 'y1': 559, 'y2': 605},
  {'x1': 539, 'x2': 576, 'y1': 687, 'y2': 646},
  {'x1': 476, 'x2': 514, 'y1': 625, 'y2': 570}],
 'BubbleSortresp(WIC)': [{'x1': 809, 'x2': 842, 'y1': 185, 'y2': 231},
  {'x1': 1104, 'x2': 1143, 'y1': 251, 'y2': 298},
  {'x1': 1040, 'x2': 1074, 'y1': 324, 'y2': 362},
  {'x1': 1205, 'x2': 1238, 'y1': 357, 'y2': 395},
  {'x1': 1223, 'x2': 1256, 'y1': 395, 'y2': 433},
  {'x1': 670, 'x2': 711, 'y1': 525, 'y2': 571},
  {'x1': 604, 'x2': 644, 'y1': 561, 'y2': 608},
  {'x1': 539, 'x2': 577, 'y1': 590, 'y2': 641},
  {'x1': 476, 'x2': 514, 'y1': 628, 'y2': 643},
  {'x1': 1120, 'x2': 1157, 'y1': 692, 'y2': 739},
  {'x1': 475, 'x2': 510, 'y1': 831, 'y2': 863},
  {'x1': 409, 'x2': 447, 'y1': 863, 'y2': 912}],
 'BubbleSort(CNL)': [{'x1': 585, 'x2': 518, 'y1': 194, 'y2': 229},
  {'x1': 541, 'x2': 573, 'y1': 253, 'y2': 288},
  {'x1': 595, 'x2': 630, 'y1': 338, 'y2': 376},
  {'x1': 651, 'x2': 688, 'y1': 398, 'y2': 435},
  {'x1': 704, 'x2': 749, 'y1': 457, 'y2': 493},
  {'x1': 707, 'x2': 747, 'y1': 570, 'y2': 615},
  {'x1': 650, 'x2': 690, 'y1': 600, 'y2': 642},
  {'x1': 599, 'x2': 631, 'y1': 625, 'y2': 672},
  {'x1': 539, 'x2': 576, 'y1': 660, 'y2': 703}],
 'BubbleSortresp(CNL)': [{'x1': 483, 'x2': 520, 'y1': 194, 'y2': 231},
  {'x1': 538, 'x2': 577, 'y1': 253, 'y2': 286},
  {'x1': 651, 'x2': 692, 'y1': 398, 'y2': 436},
  {'x1': 707, 'x2': 752, 'y1': 458, 'y2': 495},
  {'x1': 706, 'x2': 747, 'y1': 572, 'y2': 610},
  {'x1': 656, 'x2': 682, 'y1': 593, 'y2': 635},
  {'x1': 592, 'x2': 633, 'y1': 626, 'y2': 677},
  {'x1': 538, 'x2': 571, 'y1': 656, 'y2': 702},
  {'x1': 537, 'x2': 577, 'y1': 751, 'y2': 787},
  {'x1': 538, 'x2': 576, 'y1': 858, 'y2': 904},
  {'x1': 480, 'x2': 519, 'y1': 889, 'y2': 940}],
 'NumberCheck(WOC)': [{'x1': 639, 'x2': 674, 'y1': 203, 'y2': 241},
  {'x1': 707, 'x2': 740, 'y1': 270, 'y2': 311},
  {'x1': 470, 'x2': 504, 'y1': 331, 'y2': 372},
  {'x1': 874, 'x2': 905, 'y1': 361, 'y2': 411},
  {'x1': 468, 'x2': 504, 'y1': 427, 'y2': 472},
  {'x1': 417, 'x2': 450, 'y1': 461, 'y2': 506}],
 'NumberCheckresp(WOC)': [{'x1': 640, 'x2': 673, 'y1': 203, 'y2': 247},
  {'x1': 708, 'x2': 743, 'y1': 273, 'y2': 309},
  {'x1': 469, 'x2': 503, 'y1': 335, 'y2': 372},
  {'x1': 470, 'x2': 502, 'y1': 425, 'y2': 473},
  {'x1': 421, 'x2': 446, 'y1': 459, 'y2': 508},
  {'x1': 765, 'x2': 795, 'y1': 521, 'y2': 567},
  {'x1': 994, 'x2': 1033, 'y1': 588, 'y2': 632},
  {'x1': 469, 'x2': 505, 'y1': 813, 'y2': 861},
  {'x1': 875, 'x2': 905, 'y1': 363, 'y2': 408},
  {'x1': 415, 'x2': 449, 'y1': 843, 'y2': 892}],
 'NumberCheck(WIC)': [{'x1': 667, 'x2': 687, 'y1': 191, 'y2': 231},
  {'x1': 724, 'x2': 752, 'y1': 254, 'y2': 292},
  {'x1': 498, 'x2': 532, 'y1': 316, 'y2': 355},
  {'x1': 875, 'x2': 907, 'y1': 339, 'y2': 386},
  {'x1': 497, 'x2': 530, 'y1': 404, 'y2': 451},
  {'x1': 446, 'x2': 881, 'y1': 432, 'y2': 474}],
 'NumberCheckresp(WIC)': [{'x1': 657, 'x2': 685, 'y1': 195, 'y2': 233},
  {'x1': 723, 'x2': 750, 'y1': 256, 'y2': 290},
  {'x1': 497, 'x2': 531, 'y1': 315, 'y2': 350},
  {'x1': 878, 'x2': 906, 'y1': 343, 'y2': 335},
  {'x1': 448, 'x2': 530, 'y1': 404, 'y2': 448},
  {'x1': 448, 'x2': 479, 'y1': 335, 'y2': 475},
  {'x1': 475, 'x2': 803, 'y1': 497, 'y2': 434},
  {'x1': 992, 'x2': 1025, 'y1': 553, 'y2': 599},
  {'x1': 956, 'x2': 985, 'y1': 653, 'y2': 648},
  {'x1': 550, 'x2': 583, 'y1': 709, 'y2': 745},
  {'x1': 620, 'x2': 647, 'y1': 742, 'y2': 772},
  {'x1': 549, 'x2': 579, 'y1': 797, 'y2': 833},
  {'x1': 498, 'x2': 530, 'y1': 828, 'y2': 870},
  {'x1': 448, 'x2': 481, 'y1': 854, 'y2': 904}],
 'NumberCheck(CNL)': [{'x1': 517, 'x2': 557, 'y1': 193, 'y2': 232},
  {'x1': 562, 'x2': 595, 'y1': 273, 'y2': 303},
  {'x1': 563, 'x2': 590, 'y1': 323, 'y2': 354},
  {'x1': 567, 'x2': 595, 'y1': 375, 'y2': 405},
  {'x1': 561, 'x2': 593, 'y1': 421, 'y2': 460},
  {'x1': 518, 'x2': 547, 'y1': 449, 'y2': 484}],
 'NumberCheckresp(CNL)': [{'x1': 521, 'x2': 552, 'y1': 193, 'y2': 222},
  {'x1': 567, 'x2': 591, 'y1': 270, 'y2': 301},
  {'x1': 562, 'x2': 592, 'y1': 320, 'y2': 352},
  {'x1': 565, 'x2': 595, 'y1': 377, 'y2': 403},
  {'x1': 564, 'x2': 592, 'y1': 422, 'y2': 460},
  {'x1': 519, 'x2': 545, 'y1': 449, 'y2': 487},
  {'x1': 518, 'x2': 552, 'y1': 531, 'y2': 563},
  {'x1': 565, 'x2': 596, 'y1': 582, 'y2': 611},
  {'x1': 605, 'x2': 638, 'y1': 684, 'y2': 719},
  {'x1': 607, 'x2': 636, 'y1': 736, 'y2': 771},
  {'x1': 608, 'x2': 637, 'y1': 789, 'y2': 818},
  {'x1': 604, 'x2': 638, 'y1': 840, 'y2': 873},
  {'x1': 561, 'x2': 693, 'y1': 863, 'y2': 900},
  {'x1': 519, 'x2': 549, 'y1': 889, 'y2': 927}],
 'ContainsSubstring(WOC)': [{'x1': 822, 'x2': 849, 'y1': 184, 'y2': 222},
  {'x1': 1400, 'x2': 1427, 'y1': 240, 'y2': 280},
  {'x1': 1028, 'x2': 1058, 'y1': 303, 'y2': 334},
  {'x1': 1151, 'x2': 1184, 'y1': 327, 'y2': 366},
  {'x1': 671, 'x2': 699, 'y1': 473, 'y2': 502},
  {'x1': 1095, 'x2': 1127, 'y1': 492, 'y2': 537},
  {'x1': 634, 'x2': 669, 'y1': 579, 'y2': 622},
  {'x1': 578, 'x2': 614, 'y1': 609, 'y2': 650},
  {'x1': 523, 'x2': 548, 'y1': 639, 'y2': 678},
  {'x1': 468, 'x2': 501, 'y1': 666, 'y2': 705},
  {'x1': 419, 'x2': 444, 'y1': 721, 'y2': 767}],
 'ContainsSubstringresp(WOC)': [{'x1': 820, 'x2': 848, 'y1': 184, 'y2': 233},
  {'x1': 1397, 'x2': 1436, 'y1': 234, 'y2': 285},
  {'x1': 1028, 'x2': 1056, 'y1': 301, 'y2': 334},
  {'x1': 1153, 'x2': 1179, 'y1': 326, 'y2': 365},
  {'x1': 671, 'x2': 695, 'y1': 474, 'y2': 503},
  {'x1': 1097, 'x2': 1124, 'y1':  496, 'y2': 534},
  {'x1': 635, 'x2': 672, 'y1': 579, 'y2': 621},
  {'x1': 583, 'x2': 613, 'y1': 611, 'y2': 653},
  {'x1': 524, 'x2': 560, 'y1': 637, 'y2': 681},
  {'x1': 469, 'x2': 497, 'y1': 663, 'y2': 701},
  {'x1': 416, 'x2': 446, 'y1': 724, 'y2': 765},
  {'x1': 961, 'x2': 988, 'y1': 778, 'y2': 823},
  {'x1': 414, 'x2': 438, 'y1': 891, 'y2': 930},
  {'x1': 336, 'x2': 383, 'y1': 922, 'y2': 954}],
 'ContainsSubstring(WIC)': [{'x1': 798, 'x2': 822, 'y1': 152, 'y2': 184},
  {'x1': 1393, 'x2': 1419, 'y1': 207, 'y2': 242},
  {'x1': 1027, 'x2': 1048, 'y1': 269, 'y2': 289},
  {'x1': 1146, 'x2': 1172, 'y1': 291, 'y2': 323},
  {'x1': 589, 'x2': 616, 'y1': 375, 'y2': 398},
  {'x1': 1227, 'x2': 1251, 'y1': 401, 'y2': 427},
  {'x1': 593, 'x2': 612, 'y1': 556, 'y2': 478},
  {'x1': 1091, 'x2': 1125, 'y1': 489, 'y2': 511},
  {'x1': 641, 'x2': 675, 'y1': 566, 'y2': 599},
  {'x1': 590, 'x2': 625, 'y1': 595, 'y2': 641},
  {'x1': 539, 'x2': 572, 'y1': 623, 'y2': 664},
  {'x1': 482, 'x2': 510, 'y1': 651, 'y2': 676},
  {'x1': 433, 'x2': 461, 'y1': 707, 'y2': 749},
  {'x1': 961, 'x2': 986, 'y1': 323, 'y2': 355},
  {'x1': 361, 'x2': 1013, 'y1': 516, 'y2': 593}],
 'ContainsSubstringresp(WIC)': [{'x1': 497, 'x2': 819, 'y1': 156, 'y2': 180},
 {'x1': 1388, 'x2': 11428, 'y1': 206, 'y2': 256},
 {'x1': 1026, 'x2': 1058, 'y1': 269, 'y2': 299},
 {'x1': 1147, 'x2': 1184, 'y1': 295, 'y2': 328},
 {'x1': 960, 'x2': 992, 'y1': 325, 'y2': 356},
 {'x1': 589, 'x2': 634, 'y1': 376, 'y2': 406},
 {'x1': 1229, 'x2': 1262, 'y1': 403, 'y2': 442},
 {'x1': 592, 'x2': 622, 'y1': 456, 'y2': 493},
 {'x1': 689, 'x2': 719, 'y1': 463, 'y2': 492},
 {'x1': 1090, 'x2': 1124, 'y1': 483, 'y2': 527},
 {'x1': 646, 'x2': 679, 'y1': 565, 'y2': 614},
 {'x1': 592, 'x2': 624, 'y1': 596, 'y2': 638},
 {'x1': 534, 'x2': 572, 'y1': 625, 'y2': 661},
 {'x1': 486, 'x2': 518, 'y1': 655, 'y2': 885},
 {'x1': 428, 'x2': 461, 'y1': 704, 'y2': 749},
 {'x1': 958, 'x2': 992, 'y1': 760, 'y2': 804},
 {'x1': 428, 'x2': 463, 'y1': 872, 'y2': 913},
 {'x1': 376, 'x2': 409, 'y1': 879, 'y2': 946}],
 'ContainsSubstring(CNL)': [{'x1': 475, 'x2': 509, 'y1': 180, 'y2': 185},
 {'x1': 520, 'x2': 547, 'y1': 198, 'y2': 232},
 {'x1': 569, 'x2': 594, 'y1': 271, 'y2': 304},
 {'x1': 611, 'x2': 642, 'y1': 323, 'y2': 351},
 {'x1': 658, 'x2': 686, 'y1': 369, 'y2': 394},
 {'x1': 656, 'x2': 686, 'y1': 412, 'y2': 442},
 {'x1': 655, 'x2': 685, 'y1': 461, 'y2': 492},
 {'x1': 658, 'x2': 679, 'y1': 507, 'y2': 534},
 {'x1': 660, 'x2': 685, 'y1': 535, 'y2': 559},
 {'x1': 699, 'x2': 810, 'y1': 577, 'y2': 734},
 {'x1': 702, 'x2': 724, 'y1': 646, 'y2': 681},
 {'x1': 658, 'x2': 684, 'y1': 672, 'y2': 700},
 {'x1': 613, 'x2': 636, 'y1': 694, 'y2': 725},
 {'x1': 565, 'x2': 594, 'y1': 716, 'y2': 749},
 {'x1': 523, 'x2': 554, 'y1': 765, 'y2': 795}],
 'ContainsSubstringresp(CNL)': [{'x1': 474, 'x2': 502, 'y1': 156, 'y2': 184},
 {'x1': 521, 'x2': 548, 'y1': 206, 'y2': 230},
 {'x1': 566, 'x2': 594, 'y1': 272, 'y2': 301},
 {'x1': 610, 'x2': 639, 'y1': 320, 'y2': 347},
 {'x1': 654, 'x2': 686, 'y1': 369, 'y2': 396},
 {'x1': 657, 'x2': 685, 'y1': 416, 'y2': 441},
 {'x1': 660, 'x2': 680, 'y1': 460, 'y2': 480},
 {'x1': 659, 'x2': 678, 'y1': 508, 'y2': 533},
 {'x1': 661, 'x2': 683, 'y1': 533, 'y2': 556},
 {'x1': 702, 'x2': 729, 'y1': 579, 'y2': 605},
 {'x1': 702, 'x2': 729, 'y1': 648, 'y2': 675},
 {'x1': 657, 'x2': 684, 'y1': 670, 'y2': 705},
 {'x1': 361, 'x2': 1013, 'y1': 516, 'y2': 593},
 {'x1': 610, 'x2': 635, 'y1': 695, 'y2': 724},
 {'x1': 566, 'x2': 591, 'y1': 717, 'y2': 746},
 {'x1': 522, 'x2': 547, 'y1': 766, 'y2': 794},
 {'x1': 523, 'x2': 546, 'y1': 837, 'y2': 861},
 {'x1': 521, 'x2': 543, 'y1': 837, 'y2': 861},
 {'x1': 478, 'x2': 498, 'y1': 954, 'y2': 988}],
 'Power(WOC)': [{'x1': 699, 'x2': 749, 'y1': 230, 'y2': 283},
 {'x1': 1370, 'x2': 1416, 'y1': 315, 'y2': 366},
 {'x1': 389, 'x2': 432, 'y1': 523, 'y2': 576}],
 'Powerresp(WOC)': [{'x1': 701, 'x2': 736, 'y1': 228, 'y2': 276},
 {'x1': 1373, 'x2': 1415, 'y1': 315, 'y2': 367},
 {'x1': 395, 'x2': 423, 'y1': 522, 'y2': 552},
 {'x1': 1100, 'x2': 1231, 'y1': 611, 'y2': 654},
 {'x1': 393, 'x2': 431, 'y1': 774, 'y2': 828},
 {'x1': 315, 'x2': 351, 'y1': 816, 'y2': 874}],
 'Power(WIC)': [{'x1': 709, 'x2': 751, 'y1': 222, 'y2': 277},
 {'x1': 1358, 'x2': 1398, 'y1': 300, 'y2': 356},
 {'x1': 1262, 'x2': 1299, 'y1': 385, 'y2': 430},
 {'x1': 493, 'x2': 529, 'y1': 467, 'y2': 509},
 {'x1': 413, 'x2': 453, 'y1': 547, 'y2': 595}],
 'Powerresp(WIC)': [{'x1': 714, 'x2': 744, 'y1': 222, 'y2': 270},
 {'x1': 1359, 'x2': 1397, 'y1': 306, 'y2': 350},
 {'x1': 1260, 'x2': 1296, 'y1': 387, 'y2': 431},
 {'x1': 492, 'x2': 528, 'y1': 467, 'y2': 511},
 {'x1': 415, 'x2': 438, 'y1': 546, 'y2': 570},
 {'x1': 1183, 'x2': 1218, 'y1': 626, 'y2': 676},
 {'x1': 416, 'x2': 457, 'y1': 789, 'y2': 841},
 {'x1': 340, 'x2': 374, 'y1': 828, 'y2': 877}],
 'Power(CNL)': [{'x1': 432, 'x2': 465, 'y1': 267, 'y2': 311},
 {'x1': 493, 'x2': 516, 'y1': 343, 'y2': 355},
 {'x1': 493, 'x2': 531, 'y1': 525, 'y2': 570}],
 'Powerresp(CNL)': [{'x1': 429, 'x2': 466, 'y1': 273, 'y2': 305},
 {'x1': 494, 'x2': 530, 'y1': 345, 'y2': 381},
 {'x1': 491, 'x2': 533, 'y1': 526, 'y2': 565},
 {'x1': 494, 'x2': 523, 'y1': 640, 'y2': 673},
 {'x1': 498, 'x2': 524, 'y1': 778, 'y2': 828},
 {'x1': 435, 'x2': 461, 'y1': 821, 'y2': 865},
 {'x1': 361, 'x2': 1013, 'y1': 516, 'y2': 593}],
 'BinaryConversion(WOC)': [{'x1': 908, 'x2': 935, 'y1': 210, 'y2': 256},
 {'x1': 979, 'x2': 1005, 'y1': 282, 'y2': 319},
 {'x1': 784, 'x2': 815, 'y1': 391, 'y2': 423},
 {'x1': 570, 'x2': 605, 'y1': 525, 'y2': 563},
 {'x1': 508, 'x2': 538, 'y1': 627, 'y2': 673}],
 'BinaryConversionresp(WOC)': [{'x1': 905, 'x2': 930, 'y1': 215, 'y2': 237},
  {'x1': 979, 'x2': 1009, 'y1': 287, 'y2': 320},
  {'x1': 784, 'x2': 813, 'y1': 390, 'y2': 424},
  {'x1': 569, 'x2': 599, 'y1': 527, 'y2': 565},
  {'x1': 507, 'x2': 542, 'y1': 632, 'y2': 672},
  {'x1': 1102, 'x2': 1130, 'y1': 969, 'y2': 743},
  {'x1': 511, 'x2': 541, 'y1': 803, 'y2': 844},
  {'x1': 452, 'x2': 476, 'y1': 836, 'y2': 872}],
 'BinaryConversion(WIC)': [{'x1': 906, 'x2': 935, 'y1': 209, 'y2': 248},
  {'x1': 976, 'x2': 1013, 'y1': 274, 'y2': 312},
  {'x1': 788, 'x2': 818, 'y1': 378, 'y2': 410},
  {'x1': 578, 'x2': 609, 'y1': 511, 'y2': 548},
  {'x1': 1066, 'x2': 1094, 'y1': 543, 'y2': 582},
  {'x1': 579, 'x2': 612, 'y1': 612, 'y2': 651},
  {'x1': 521, 'x2': 556, 'y1': 643, 'y2': 683}],
 'BinaryConversionresp(WIC)': [{'x1': 907, 'x2': 934, 'y1': 204, 'y2': 249},
  {'x1': 975, 'x2': 1007, 'y1': 276, 'y2': 310},
  {'x1': 787, 'x2': 818, 'y1': 348, 'y2': 411},
  {'x1': 582, 'x2': 612, 'y1': 511, 'y2': 547},
  {'x1': 1064, 'x2': 1095, 'y1': 542, 'y2': 579},
  {'x1': 579, 'x2': 612, 'y1': 608, 'y2': 652},
  {'x1': 521, 'x2': 555, 'y1': 642, 'y2': 688},
  {'x1': 1094, 'x2': 1130, 'y1': 708, 'y2': 752},
  {'x1': 523, 'x2': 553, 'y1': 807, 'y2': 850},
  {'x1': 467, 'x2': 493, 'y1': 843, 'y2': 885}],
 'BinaryConversion(CNL)': [{'x1': 519, 'x2': 549, 'y1': 215, 'y2': 252},
  {'x1': 574, 'x2': 600, 'y1': 275, 'y2': 311},
  {'x1': 524, 'x2': 556, 'y1': 393, 'y2': 433},
  {'x1': 619, 'x2': 649, 'y1': 514, 'y2': 546},
  {'x1': 627, 'x2': 655, 'y1': 570, 'y2': 610},
  {'x1': 621, 'x2': 654, 'y1': 627, 'y2': 668},
  {'x1': 573, 'x2': 600, 'y1': 660, 'y2': 700}],
  'BinaryConversionresp(CNL)': [{'x1': 524, 'x2': 552, 'y1': 214, 'y2': 249},
   {'x1': 570, 'x2': 602, 'y1': 277, 'y2': 308},
   {'x1': 622, 'x2': 664, 'y1': 392, 'y2': 429},
   {'x1': 620, 'x2': 651, 'y1': 514, 'y2': 549},
   {'x1': 623, 'x2': 660, 'y1': 575, 'y2': 618},
   {'x1': 622, 'x2': 651, 'y1': 630, 'y2': 671},
   {'x1': 575, 'x2': 598, 'y1': 660, 'y2': 699},
   {'x1': 579, 'x2': 597, 'y1': 752, 'y2': 786},
   {'x1': 568, 'x2': 600, 'y1': 841, 'y2': 876},
   {'x1': 361, 'x2': 1013, 'y1': 516, 'y2': 593},
   {'x1': 517, 'x2': 550, 'y1': 869, 'y2': 910}],
 'MultiplyMatrix(WOC)': [{'x1': 777, 'x2': 806, 'y1': 174, 'y2': 212},
   {'x1': 1384, 'x2': 1312, 'y1': 230, 'y2': 268},
   {'x1': 828, 'x2': 853, 'y1': 290, 'y2': 319},
   {'x1': 878, 'x2': 903, 'y1': 320, 'y2': 347},
   {'x1': 578, 'x2': 608, 'y1': 431, 'y2': 464},
   {'x1': 529, 'x2': 556, 'y1': 458, 'y2': 493},
   {'x1': 482, 'x2': 510, 'y1': 484, 'y2': 522}],
 'MultiplyMatrixresp(WOC)': [{'x1': 779, 'x2': 803, 'y1': 172, 'y2': 210},
    {'x1': 1284, 'x2': 1309, 'y1': 233, 'y2': 267},
    {'x1': 828, 'x2': 851, 'y1': 289, 'y2': 319},
    {'x1': 877, 'x2': 901, 'y1': 318, 'y2': 352},
    {'x1': 577, 'x2': 604, 'y1': 431, 'y2': 462},
    {'x1': 528, 'x2': 557, 'y1': 457, 'y2': 492},
    {'x1': 481, 'x2': 504, 'y1': 486, 'y2': 517},
    {'x1': 963, 'x2': 990, 'y1': 940, 'y2': 571},
    {'x1': 829, 'x2': 852, 'y1': 715, 'y2': 742},
    {'x1': 530, 'x2': 557, 'y1': 824, 'y2': 860},
    {'x1': 442, 'x2': 503, 'y1': 854, 'y2': 882},
    {'x1': 433, 'x2': 458, 'y1': 883, 'y2': 913}],
 'MultiplyMatrix(WIC)': [{'x1': 792, 'x2': 817, 'y1': 165, 'y2': 201},
    {'x1': 1272, 'x2': 1298, 'y1': 216, 'y2': 257},
    {'x1': 839, 'x2': 867, 'y1': 277, 'y2': 304},
    {'x1': 887, 'x2': 910, 'y1': 229, 'y2': 335},
    {'x1': 935, 'x2': 935, 'y1': 357, 'y2': 381},
    {'x1': 653, 'x2': 675, 'y1': 409, 'y2': 440},
    {'x1': 604, 'x2': 628, 'y1': 438, 'y2': 466},
    {'x1': 556, 'x2': 583, 'y1': 462, 'y2': 499},
    {'x1': 510, 'x2': 535, 'y1': 489, 'y2': 523}],
'MultiplyMatrixresp(WIC)': [{'x1': 793, 'x2': 818, 'y1': 164, 'y2': 198},
   {'x1': 1275, 'x2': 1298, 'y1': 222, 'y2': 251},
   {'x1': 841, 'x2': 862, 'y1': 278, 'y2': 302},
   {'x1': 886, 'x2': 910, 'y1': 304, 'y2': 331},
   {'x1': 935, 'x2': 957, 'y1': 335, 'y2': 382},
   {'x1': 651, 'x2': 671, 'y1': 409, 'y2': 437},
   {'x1': 603, 'x2': 627, 'y1': 436, 'y2': 465},
   {'x1': 559, 'x2': 575, 'y1': 464, 'y2': 488},
   {'x1': 510, 'x2': 531, 'y1': 490, 'y2': 516},
   {'x1': 976, 'x2': 991, 'y1': 554, 'y2': 568},
   {'x1': 838, 'x2': 864, 'y1': 709, 'y2': 731},
   {'x1': 887, 'x2': 910, 'y1': 732, 'y2': 760},
   {'x1': 600, 'x2': 625, 'y1': 785, 'y2': 811},
   {'x1': 554, 'x2': 581, 'y1': 837, 'y2': 869},
   {'x1': 510, 'x2': 532, 'y1': 864, 'y2': 893},
   {'x1': 863, 'x2': 485, 'y1': 894, 'y2': 922}],
 'MultiplyMatrix(CNL)': [{'x1': 542, 'x2': 565, 'y1': 166, 'y2': 191},
    {'x1': 579, 'x2': 606, 'y1': 211, 'y2': 236},
    {'x1': 619, 'x2': 646, 'y1': 280, 'y2':309},
    {'x1': 660, 'x2': 683, 'y1': 325, 'y2': 352},
    {'x1': 699, 'x2': 723, 'y1': 394, 'y2': 419},
    {'x1': 697, 'x2': 724, 'y1': 442, 'y2': 465},
    {'x1': 661, 'x2': 681, 'y1': 642, 'y2': 488},
    {'x1': 620, 'x2': 642, 'y1': 485, 'y2': 512},
    {'x1': 582, 'x2': 603, 'y1': 507, 'y2': 537}],
 'MultiplyMatrixresp(CNL)': [{'x1': 543, 'x2': 561, 'y1': 166, 'y2': 192},
    {'x1': 581, 'x2': 604, 'y1': 212, 'y2': 237},
    {'x1': 620, 'x2': 647, 'y1': 282, 'y2': 307},
    {'x1': 658, 'x2': 687, 'y1': 327, 'y2': 351},
    {'x1': 697, 'x2': 726, 'y1': 394, 'y2': 420},
    {'x1': 703, 'x2': 726, 'y1': 441, 'y2': 468},
    {'x1': 661, 'x2': 684, 'y1': 464, 'y2': 489},
    {'x1': 621, 'x2': 643, 'y1': 484, 'y2': 511},
    {'x1': 580, 'x2': 603, 'y1': 509, 'y2': 536},
    {'x1': 581, 'x2': 604, 'y1': 580, 'y2': 602},
    {'x1': 619, 'x2': 645, 'y1': 738, 'y2': 762},
    {'x1': 661, 'x2': 682, 'y1': 782, 'y2': 809},
    {'x1': 662, 'x2': 684, 'y1': 828, 'y2': 853},
    {'x1': 623, 'x2': 642, 'y1': 874, 'y2': 900},
    {'x1': 583, 'x2': 604, 'y1': 996, 'y2': 924},
    {'x1': 543, 'x2': 565, 'y1': 916, 'y2': 948},
    {'x1': 361, 'x2': 1013, 'y1': 516, 'y2': 593},
    {'x1': 361, 'x2': 1013, 'y1': 516, 'y2': 593},
    {'x1': 361, 'x2': 1013, 'y1': 516, 'y2': 593}],
 'PrimeNumber(WOC)': [{'x1': 781, 'x2': 810, 'y1': 150, 'y2': 190},
   {'x1': 891, 'x2': 990, 'y1': 215, 'y2': 257},
   {'x1': 557, 'x2': 591, 'y1': 383, 'y2': 417},
   {'x1': 828, 'x2': 857, 'y1': 416, 'y2': 449},
   {'x1': 779, 'x2': 806, 'y1': 449, 'y2': 484},
   {'x1': 598, 'x2': 936, 'y1': 579, 'y2': 612},
   {'x1': 537, 'x2': 567, 'y1': 610, 'y2': 648},
   {'x1': 471, 'x2': 500, 'y1': 708, 'y2': 746},
   {'x1': 411, 'x2': 439, 'y1': 742, 'y2': 764}],
   'PrimeNumberresp(WOC)': [{'x1': 779, 'x2': 810, 'y1': 149, 'y2': 192},
   {'x1': 890, 'x2': 923, 'y1': 213, 'y2': 254},
   {'x1': 827, 'x2': 860, 'y1': 411, 'y2': 454},
   {'x1': 600, 'x2': 634, 'y1': 578, 'y2': 618},
   {'x1': 539, 'x2': 566, 'y1': 612, 'y2': 644},
   {'x1': 477, 'x2': 504, 'y1': 707, 'y2': 751},
   {'x1': 410, 'x2': 438, 'y1': 741, 'y2': 781},
   {'x1': 1037, 'x2': 1067, 'y1': 806, 'y2': 850},
   {'x1': 409, 'x2': 441, 'y1': 873, 'y2': 912},
   {'x1': 345, 'x2': 381, 'y1': 905, 'y2': 947}],
 'PrimeNumber(WIC)': [{'x1': 788, 'x2': 825, 'y1': 135, 'y2': 184},
     {'x1': 894, 'x2': 924, 'y1': 201, 'y2': 242},
     {'x1': 729, 'x2': 758, 'y1': 298, 'y2': 331},
     {'x1': 501, 'x2': 528, 'y1': 358, 'y2': 394},
     {'x1': 579, 'x2': 606, 'y1': 390, 'y2': 423},
     {'x1': 836, 'x2': 864, 'y1': 421, 'y2': 459},
     {'x1': 789, 'x2': 818, 'y1': 453, 'y2': 487},
     {'x1': 616, 'x2': 654, 'y1': 573, 'y2': 614},
     {'x1': 561, 'x2': 594, 'y1': 605, 'y2': 639},
     {'x1': 742, 'x2': 780, 'y1': 637, 'y2': 573},
     {'x1': 556, 'x2': 697, 'y1': 697, 'y2': 741},
     {'x1': 500, 'x2': 532, 'y1': 729, 'y2': 771},
     {'x1': 441, 'x2': 469, 'y1': 458, 'y2': 801}],
 'PrimeNumberresp(WIC)': [{'x1': 788, 'x2': 818, 'y1': 138, 'y2': 179},
     {'x1': 893, 'x2': 915, 'y1': 201, 'y2': 224},
     {'x1': 728, 'x2': 755, 'y1': 294, 'y2': 332},
     {'x1': 501, 'x2': 529, 'y1': 357, 'y2': 392},
     {'x1': 580, 'x2': 610, 'y1': 388, 'y2': 422},
     {'x1': 833, 'x2': 867, 'y1': 423, 'y2': 456},
     {'x1': 785, 'x2': 815, 'y1': 454, 'y2': 486},
     {'x1': 620, 'x2': 652, 'y1': 574, 'y2': 615},
     {'x1': 561, 'x2': 594, 'y1': 606, 'y2': 640},
     {'x1': 744, 'x2': 777, 'y1': 639, 'y2': 674},
     {'x1': 558, 'x2': 539, 'y1': 698, 'y2': 737},
     {'x1': 494, 'x2': 532, 'y1': 734, 'y2': 772},
     {'x1': 440, 'x2': 467, 'y1': 760, 'y2': 804},
     {'x1': 1028, 'x2': 1062, 'y1': 821, 'y2': 859},
     {'x1': 441, 'x2': 471, 'y1': 882, 'y2': 930},
     {'x1': 382, 'x2': 409, 'y1': 915, 'y2': 956}],
 'PrimeNumber(CNL)': [{'x1': 501, 'x2': 523, 'y1': 161, 'y2': 192},
     {'x1': 548, 'x2': 571, 'y1': 211, 'y2': 241},
     {'x1': 593, 'x2': 619, 'y1': 310, 'y2': 338},
     {'x1': 595, 'x2': 617, 'y1': 357, 'y2': 387},
     {'x1': 596, 'x2': 619, 'y1': 408, 'y2': 437},
     {'x1': 643, 'x2': 669, 'y1': 457, 'y2': 486},
     {'x1': 687, 'x2': 716, 'y1': 506, 'y2': 535},
     {'x1': 691, 'x2': 713, 'y1': 626, 'y2': 663},
     {'x1': 641, 'x2': 666, 'y1': 653, 'y2': 679},
     {'x1': 644, 'x2': 667, 'y1': 704, 'y2': 731},
     {'x1': 642, 'x2': 667, 'y1': 749, 'y2': 784},
     {'x1': 594, 'x2': 616, 'y1': 776, 'y2': 808},
     {'x1': 547, 'x2': 570, 'y1': 800, 'y2': 833}],
'PrimeNumberresp(CNL)': [{'x1': 500, 'x2':522, 'y1': 161, 'y2': 190},
   {'x1': 547, 'x2': 570, 'y1': 210, 'y2': 239},
   {'x1': 594, 'x2': 618, 'y1': 307, 'y2': 337},
   {'x1': 591, 'x2': 620, 'y1': 357, 'y2': 385},
   {'x1': 595, 'x2': 618, 'y1': 408, 'y2': 436},
   {'x1': 643, 'x2': 667, 'y1': 457, 'y2': 485},
   {'x1': 691, 'x2': 717, 'y1': 506, 'y2': 535},
   {'x1': 689, 'x2': 716, 'y1': 629, 'y2': 660},
   {'x1': 642, 'x2': 667, 'y1': 651, 'y2': 682},
   {'x1': 645, 'x2': 669, 'y1': 703, 'y2': 733},
   {'x1': 644, 'x2': 668, 'y1': 748, 'y2': 782},
   {'x1': 592, 'x2': 620, 'y1': 778, 'y2': 808},
   {'x1': 445, 'x2': 571, 'y1': 799, 'y2': 833},
   {'x1': 548, 'x2': 574, 'y1': 875, 'y2': 904},
   {'x1': 551, 'x2': 571, 'y1': 921, 'y2': 956},
   {'x1': 502, 'x2': 522, 'y1': 946, 'y2': 982 }] }


finalexport = []
for userIndex in range(len(userInfo)):
  session_number = '001'
  participant_name = userInfo[userIndex]
  print('started'+participant_name)
  df_eyetracking = pd.read_csv("./data/eye_tracking_data_" + session_number + " _ " + participant_name + ".csv", sep=";")
  df_eyetracking.columns = ["l_valid", "r_valid", "l_display", "r_display", "timestamp", "l_diameter", "r_diameter"]
  df_answers = pd.read_csv("./data/study_answers_" + session_number + " _ " + participant_name + ".csv")
  df_answers = df_answers.iloc[:, 1:]
  df_cutting = pd.read_csv("./data/study_cutting_data_" + session_number + " _ " + participant_name + ".csv")
  df_cutting = df_cutting.iloc[:, 1:]

  df_eyetracking["l_display_x"] = df_eyetracking["l_display"].apply(lambda row: int(float(re.findall("\d+\.\d+", row)[0])*screen_width) if len(re.findall("\d+\.\d+", row)) == 2 else 0.0)
  df_eyetracking["l_display_y"] = df_eyetracking["l_display"].apply(lambda row: int(float(re.findall("\d+\.\d+", row)[1])*screen_height) if len(re.findall("\d+\.\d+", row)) == 2 else 0.0)
  df_eyetracking["r_display_x"] = df_eyetracking["r_display"].apply(lambda row: int(float(re.findall("\d+\.\d+", row)[0])*screen_width) if len(re.findall("\d+\.\d+", row)) == 2 else 0.0)
  df_eyetracking["r_display_y"] = df_eyetracking["r_display"].apply(lambda row: int(float(re.findall("\d+\.\d+", row)[1])*screen_height) if len(re.findall("\d+\.\d+", row)) == 2 else 0.0)
  df_eyetracking = df_eyetracking.drop("l_display", axis=1)
  df_eyetracking = df_eyetracking.drop("r_display", axis=1)

  eye_start = df_eyetracking.loc[0, "timestamp"]
  df_eyetracking["time"] = df_eyetracking["timestamp"] - eye_start
  df_eyetracking["time"] = df_eyetracking["time"] / 1000000

  x_values = df_eyetracking[df_eyetracking["l_valid"] == 1]["l_display_x"].values
  y_values = df_eyetracking[df_eyetracking["l_valid"] == 1]["l_display_y"].values
  l_valid = df_eyetracking

  click_data = list(zip([int(x) for x in x_values], [int(y) for y in y_values]))

  if not os.path.exists(participant_name):
    os.makedirs(participant_name)

  if not os.path.exists(participant_name+"/Heatmaps"):
    os.makedirs(participant_name+"/Heatmaps")

  if not os.path.exists(participant_name+"/Fixations"):
    os.makedirs(participant_name+"/Fixations")

  if not os.path.exists(participant_name+"/Scanpaths"):
    os.makedirs(participant_name+"/Scanpaths")
  
  
  for idx, row in df_cutting.iterrows():
      image = row["ImagePath"]
      start_time = row["StartTime"]
      end_time = row["EndTime"]
      answer = ""
      #if idx % 2 != 0:
      #  answer = df_answers.at[int(idx / 2),"Answer"]

      df_tmp = df_eyetracking[(df_eyetracking["time"] >= start_time) & (df_eyetracking["time"] <= end_time)]
      
      x_values_l = df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)]["l_display_x"].values
      y_values_l = df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)]["l_display_y"].values

      x_values_r = df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)]["r_display_x"].values
      y_values_r = df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)]["r_display_y"].values

      x_values = [ int((x_l+x_r)/2.0) for (x_l, x_r) in zip(x_values_l, x_values_r)]
      y_values = [ int((y_l+y_r)/2.0) for (y_l, y_r) in zip(y_values_l, y_values_r)]

      l_diameter = df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)]["l_diameter"].values
      r_diameter = df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)]["r_diameter"].values

      click_data = list(zip([int(x) for x in x_values], [int(y) for y in y_values]))
            
      #image = image.replace("./snippets/Final/", "")
      #imageNameforAOI = image
      print(image)
      #if image == "":
       # continue
      #if image == "Welcome.png.png":
        #continue
      
     # if image == "Welcome.png":
       # continue
        
      #if ".png" in image:
          #image = image.replace('.png', '')

      #print(image)
      
      imageNameforAOI = image
      #image = image+"_AOI.png"
      imageName = image
      
      image = PIL.Image.open(imageName)
      w, h = image.size
      
      time = df_tmp[(df_tmp["l_valid"] == 1) & (df_tmp["r_valid"] == 1)]["time"].tolist()

      Sfix, Efix= fixation_detection(x_values, y_values, time, maxdist=25, mindur=0.02)

      fixationDf = pd.DataFrame(Efix)
      tempdf = pd.DataFrame()

      tempdf
      #break
      exportArrayTemp = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
      print(len(exportArrayTemp))
      exportArrayTemp[0] = participant_name
      exportArrayTemp[1] = imageNameforAOI
      exportArrayTemp[2]= row["EndTime"] - row["StartTime"]
      exportArrayTemp[3]= answer

      #print(len(fixationDf))
      if len(fixationDf) > 0 : # and len(fixationDf) < 50:
        fixationDf.columns = ['starttime', 'endtime', 'duration', 'endx', 'endy']
        exportValue = pd.DataFrame(Sfix)
        exportValue = pd.DataFrame()
        i = 4
        imageNameforAOI=imageNameforAOI.replace('./snippets/Final/',"")
        imageNameforAOI=imageNameforAOI.replace('.png',"")
        #print(imageNameforAOI)
        print(len(AOIs[imageNameforAOI]))
        for rowValue in AOIs[imageNameforAOI]:
            
        #for rowValue in AOIs[imageNameforAOI]:
          #exportArrayTemp.append(rowValue)
            fixationAOI = calculateAOIvalueswithFixations(fixationDf, rowValue["x1"], rowValue["y1"], rowValue["x2"], rowValue["y2"])
          # firstPassDuration
            exportArrayTemp[i] = fixationAOI["firstPassDuration"]
          # secondPassDuration = 
            exportArrayTemp[i + 1] = fixationAOI["secondPassDuration"]
          # noOfFixtionsInAOI = 
            exportArrayTemp[i + 2] = fixationAOI["noOfFixtionsInAOI"]
          # totalDurationFixationInAOI = 
            exportArrayTemp[i + 3] = fixationAOI["totalDuration"]
          # revisits = 
            exportArrayTemp[i + 4] = fixationAOI["revisits"]
          #totalDuration
            exportArrayTemp[i + 5] = calculateAOIvalues(x_values, y_values, time, rowValue["x1"], rowValue["y1"], rowValue["x2"], rowValue["y2"])['totalDuration']
            i = i + 6

        #draw_fixations(Efix, (1920,1080), imagefile=imageName, durationsize=False, durationcolour=False, alpha=0.5, savefilename=participant_name+"/Fixations/Fixation_"+imageName)
        #print("Created : "+participant_name+"/Fixations/Fixation_"+imageName)
            
        #Ssac, Esac = saccade_detection(x_values, y_values, time, minlen=0.020, maxvel=20, maxacc=300)
        
        #draw_scanpath(Efix, Esac, (1920,1080), imagefile=imageName, alpha=0.5, savefilename=participant_name+"/Scanpaths/ScanPath_"+imageName)
        #print("Created : "+participant_name+"/Scanpaths/ScanPath_"+imageName)
        
      finalexport.append(exportArrayTemp)

      heatmap = gen_heatmap(click_data, w,h, 50)

      # perform gaussian blur
      cmap = plt.cm.get_cmap("jet")
      heatmap = gaussian_filter(heatmap, sigma=5)
      heatmap = cmap(heatmap)
      plt.imshow(heatmap, cmap='jet', interpolation='nearest')
      image.resize((h, w))
      image = np.array(image)
      heatmap=heatmap*255
      heatmap=heatmap.astype(int)
      heatmap_image = 0.7*image + 0.3*heatmap
      #image_name = participant_name+"/Heatmaps/Heatmap_ArraySumresp(WOC).png"
      #print("Created : "+image_name)
      im = PIL.Image.fromarray(np.uint8(heatmap_image))
      #im.save(image_name)
      #display(im)
      #plt.imshow(heatmap, cmap='jet', interpolation='nearest')
      #break
      
print(finalexport)
exportValue = pd.DataFrame(finalexport, columns=['User','Exercise','ResponseTime','Answer','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI','FirstPassDuration','SecondPassDuration','noOfFixtionsInAOI','TotalDurationFixationInAOI','Revisits','TotalDurationInAOI'])
print(exportValue.shape)
exportValue.to_csv("user8.csv")

starteduser8
./snippets/Final/ArraySum(WIC).png
124
5
./snippets/Final/ArraySumresp(WIC).png
124
7
./snippets/Final/BubbleSort(WOC).png
124
5
./snippets/Final/BubbleSortresp(WOC).png
124
8
./snippets/Final/NumberCheck(CNL).png
124
6
./snippets/Final/NumberCheckresp(CNL).png
124
14
./snippets/Final/ContainsSubstring(WIC).png
124
15
./snippets/Final/ContainsSubstringresp(WIC).png
124
18
./snippets/Final/Power(WOC).png
124
3
./snippets/Final/Powerresp(WOC).png
124
6
./snippets/Final/BinaryConversion(CNL).png
124
7
./snippets/Final/BinaryConversionresp(CNL).png
124
11
./snippets/Final/MultiplyMatrix(WIC).png
124
9
./snippets/Final/MultiplyMatrixresp(WIC).png
124
16
./snippets/Final/PrimeNumber(WOC).png
124
9
./snippets/Final/PrimeNumberresp(WOC).png
124
10


In [ ]:
#df_one = pd.read_csv("./finalexportAOI.csv", sep=",")
#df_one
#df_two = pd.read_csv("./finalexportAOI2.csv", sep=",")
#df_three = pd.read_csv("./finalexportAOI3.csv", sep=",")
#df_four = pd.read_csv("./finalexportAOI4.csv", sep=",")
#df_five = pd.read_csv("./finalexportAOI5.csv", sep=",")
#df_one =df_one.append(df_two)
#df_one =df_one.append(df_five)
#df_one =df_one.append(df_three)
#df_one =df_one.append(df_four)
#df_one.to_csv("finalValues.csv")